In [5]:
!pip install torch-geometric


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.6 MB/s eta 0:00:0000:01


In [ ]:
import torch
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv, BatchNorm
from collections import defaultdict
import json
import networkx as nx
from torch import nn
import math
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_score, train_test_split
from sklearn.metrics import f1_score, homogeneity_score, normalized_mutual_info_score, adjusted_rand_score
import shutil

In [ ]:
embedding = np.load("/Users/arjuns/Downloads/code_res_paper/paper-results-code/gnn/ds/v1/graph2vec_embeddings.npy")
labels = np.load("/Users/arjuns/Downloads/code_res_paper/paper-results-code/gnn/ds/v1/graph2vec_labels.npy")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def delete_all_contents(folder_path):
    if not os.path.exists(folder_path):
        print(f"Error: {folder_path} does not exist.")
        return
    
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        
        try:
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)  # Delete file or symlink
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)  # Delete directory and its contents
            print(f"Deleted: {item_path}")
        except Exception as e:
            print(f"Failed to delete {item_path}: {e}")

delete_all_contents("/kaggle/working/")

In [10]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True

In [12]:
class NodeClassificationGNN(nn.Module):
    def __init__(self, input_dim, hidden_channel, num_classes, dropout=0.3, activation=nn.LeakyReLU):
        super().__init__()
        self.activation = activation()  # Initialize activation function
        self.conv1 = SAGEConv(input_dim, hidden_channel)
        self.bn1 = BatchNorm(hidden_channel)  # BatchNorm for first layer
        self.conv2 = SAGEConv(hidden_channel, hidden_channel)
        self.bn2 = BatchNorm(hidden_channel)  # BatchNorm for second layer
        self.dropout = nn.Dropout(dropout)
        
        # MLP for final classification
        self.mlp = nn.Sequential(
            nn.Linear(hidden_channel, hidden_channel),
            self.activation,
            nn.Dropout(dropout),
            nn.Linear(hidden_channel, num_classes)
        )

    def forward(self, x, edge_index):
        # First convolutional layer with batch normalization and activation
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        # Second convolutional layer with batch normalization and activation
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = self.activation(x)
        x = self.dropout(x)
        
        # Pass through the MLP for final predictions
        x = self.mlp(x)
        return x


In [15]:
def build_similarity_graph(embeddings, threshold):
    num_nodes = embeddings.shape[0]
    similarity_matrix = cosine_similarity(embeddings)

    G = nx.Graph()
    for i in range(num_nodes):
        G.add_node(i, embedding=embeddings[i])

    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if similarity_matrix[i, j] >= threshold:
                G.add_edge(i, j, weight=similarity_matrix[i, j])

    return G

def convert_to_torch_geometric(graph):
    node_features = np.array([graph.nodes[n]['embedding'] for n in graph.nodes])
    edge_index = np.array(list(graph.edges)).T
    edge_weights = np.array([graph[u][v]['weight'] for u, v in graph.edges])

    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_weights = torch.tensor(edge_weights, dtype=torch.float)

    data = Data(x=x, edge_index=edge_index, edge_attr=edge_weights)

    return data

def split_data(data, labels, train_ratio=0.7, val_ratio=0.2, random_state=42):
    train_idx, temp_idx, train_labels, temp_labels = train_test_split(
        np.arange(len(labels)), labels, test_size=(1 - train_ratio), stratify=labels,  random_state=random_state
    )
    val_size = val_ratio / (1 - train_ratio)
    val_idx, test_idx, val_labels, test_labels = train_test_split(
        temp_idx, temp_labels, test_size=(1 - val_size), stratify=temp_labels, random_state=random_state
    )
    return train_idx, val_idx, test_idx

def load_best_model(model, threshold, save_path='checkpoints'):
    checkpoint = torch.load(os.path.join(save_path, f'best_model_threshold_{threshold:.2f}.pt'))
    model.load_state_dict(checkpoint['model_state_dict'])
    return checkpoint['best_val_loss']

def save_training_history(history, threshold, save_path='training_history'):
    os.makedirs(save_path, exist_ok=True)
    with open(os.path.join(save_path, f'history_threshold_{threshold:.2f}.json'), 'w') as f:
        json.dump(history, f, indent=4)

In [16]:
def train(optimizer, criterion, model, data, _, threshold, save_path='checkpoints', num_epochs=300):

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                                         factor=0.5, patience=10, 
                                                         min_lr=1e-6, verbose=True)
    
    history = defaultdict(list)
    os.makedirs(save_path, exist_ok=True)
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        pred = out.argmax(dim=1)
        correct = int(pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
        train_acc = correct / data.train_mask.sum().item()

        model.eval()
        with torch.no_grad():
            val_out = model(data.x, data.edge_index)
            val_loss = criterion(val_out[data.val_mask], data.y[data.val_mask])
            val_pred = val_out.argmax(dim=1)
            val_correct = int(val_pred[data.val_mask].eq(data.y[data.val_mask]).sum().item())
            val_acc = val_correct / data.val_mask.sum().item()

        scheduler.step(val_loss)

        history['train_loss'].append(loss.item())
        history['val_loss'].append(val_loss.item())
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        
        if val_loss.item() < best_val_loss:
            best_val_loss = val_loss.item()
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_loss': best_val_loss
            }, os.path.join(save_path, f'best_model_threshold_{threshold:.2f}.pt'))
        
        if epoch % 10 == 0:
            print(f"Epoch {epoch}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, "
                  f"Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")
        
    save_training_history(history, threshold)
    
    print(f"Best Validation Loss: {best_val_loss:.4f}")

    return best_val_loss, history


In [17]:
def build_graphs_and_setup_model_cv(embedding, labels, thresholds=np.arange(0.05, 0.16, 0.01), n_splits=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    best_threshold = None
    best_avg_test_acc = 0
    all_results = {}

    train_idx, test_idx = train_test_split(
        np.arange(len(labels)), test_size=0.3, stratify=labels, random_state=42
    )

    for threshold in thresholds:
        print(f"\nTraining with threshold: {threshold}")
        similarity_graph = build_similarity_graph(embedding, threshold=threshold)
        data = convert_to_torch_geometric(similarity_graph)
        data.y = torch.tensor(labels, dtype=torch.long).to(device)

        test_mask = torch.zeros(data.num_nodes, dtype=torch.bool).to(device)
        test_mask[test_idx] = True
        data.test_mask = test_mask
        data = data.to(device)

        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_results = []

        for fold, (train_sub_idx, val_idx) in enumerate(skf.split(train_idx, labels[train_idx])):
            print(f"  Fold {fold + 1}/{n_splits}")

            train_mask = torch.zeros(data.num_nodes, dtype=torch.bool).to(device)
            val_mask = torch.zeros(data.num_nodes, dtype=torch.bool).to(device)
            train_mask[train_idx[train_sub_idx]] = True
            val_mask[train_idx[val_idx]] = True
            data.train_mask = train_mask
            data.val_mask = val_mask

            input_dim = 128
            hidden_channels = 256
            out_ch = len(np.unique(labels))
            model = NodeClassificationGNN(
                input_dim=input_dim, hidden_channel=hidden_channels, num_classes=out_ch, dropout=0.7
            ).to(device)
            optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.1)
            criterion = torch.nn.CrossEntropyLoss()

            _, history = train(optimizer, criterion, model, data, labels, threshold)
            model.eval()

            with torch.no_grad():
                test_out = model(data.x, data.edge_index)
                test_pred = test_out.argmax(dim=1).cpu().numpy()
                test_true = data.y[data.test_mask].cpu().numpy()
                test_pred_filtered = test_pred[data.test_mask.cpu().numpy()]

                test_correct = (test_pred_filtered == test_true).sum()
                test_acc = test_correct / len(test_true)
                test_f1 = f1_score(test_true, test_pred_filtered, average='weighted')
                test_homogeneity = homogeneity_score(test_true, test_pred_filtered)
                test_nmi = normalized_mutual_info_score(test_true, test_pred_filtered)
                test_ari = adjusted_rand_score(test_true, test_pred_filtered)

            print(f"Fold Test Accuracy: {test_acc:.4f}, F1-score: {test_f1:.4f}, Homogeneity: {test_homogeneity:.4f}, "
                  f"NMI: {test_nmi:.4f}, ARI: {test_ari:.4f}")

            fold_results.append({
                'test_accuracy': test_acc,
                'f1_score': test_f1,
                'homogeneity': test_homogeneity,
                'nmi': test_nmi,
                'ari': test_ari,
            })

        avg_results = {metric: sum(fold[metric] for fold in fold_results) / n_splits for metric in fold_results[0]}
        print(f"  Average Metrics for Threshold {threshold:.2f}: {avg_results}")

        all_results[threshold] = avg_results
        if avg_results['test_accuracy'] > best_avg_test_acc:
            best_avg_test_acc = avg_results['test_accuracy']
            best_threshold = threshold

    if not os.path.exists('results'):
        os.makedirs('results')

    with open('results/cv_results.json', 'w') as f:
        json.dump({
            'best_threshold': best_threshold,
            'best_avg_test_accuracy': best_avg_test_acc,
            'all_thresholds_results': {str(k): v for k, v in all_results.items()}
        }, f, indent=4)

    print(f"\nBest Threshold: {best_threshold:.2f}, Best Average Test Accuracy: {best_avg_test_acc:.4f}")
    return all_results

In [18]:
build_graphs_and_setup_model_cv(embedding, labels)


Training with threshold: 0.05
  Fold 1/5


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.7029, Val Loss: 2.5638, Acc: 0.0760, Val Acc: 0.0824
Epoch 10, Loss: 2.4625, Val Loss: 2.4312, Acc: 0.1620, Val Acc: 0.2301
Epoch 20, Loss: 2.1954, Val Loss: 2.1549, Acc: 0.2345, Val Acc: 0.2585
Epoch 30, Loss: 1.9779, Val Loss: 1.8776, Acc: 0.2843, Val Acc: 0.3267
Epoch 40, Loss: 1.7424, Val Loss: 1.6431, Acc: 0.3276, Val Acc: 0.3892
Epoch 50, Loss: 1.5365, Val Loss: 1.4133, Acc: 0.4115, Val Acc: 0.5170
Epoch 60, Loss: 1.3705, Val Loss: 1.2065, Acc: 0.4733, Val Acc: 0.6136
Epoch 70, Loss: 1.2163, Val Loss: 1.0476, Acc: 0.5259, Val Acc: 0.6420
Epoch 80, Loss: 1.0577, Val Loss: 0.9087, Acc: 0.5714, Val Acc: 0.6761
Epoch 90, Loss: 0.9194, Val Loss: 0.7965, Acc: 0.6382, Val Acc: 0.6960
Epoch 100, Loss: 0.8325, Val Loss: 0.7263, Acc: 0.6532, Val Acc: 0.7358
Epoch 110, Loss: 0.7607, Val Loss: 0.6871, Acc: 0.7015, Val Acc: 0.7216
Epoch 120, Loss: 0.6837, Val Loss: 0.6853, Acc: 0.7264, Val Acc: 0.6960
Epoch 130, Loss: 0.6069, Val Loss: 0.6774, Acc: 0.7541, Val Acc: 0.6960
Epo

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.7067, Val Loss: 2.5618, Acc: 0.0675, Val Acc: 0.0710
Epoch 10, Loss: 2.4522, Val Loss: 2.4272, Acc: 0.1599, Val Acc: 0.2415
Epoch 20, Loss: 2.1887, Val Loss: 2.1236, Acc: 0.2416, Val Acc: 0.3523
Epoch 30, Loss: 1.9564, Val Loss: 1.8355, Acc: 0.2765, Val Acc: 0.3438
Epoch 40, Loss: 1.7493, Val Loss: 1.5720, Acc: 0.3255, Val Acc: 0.4205
Epoch 50, Loss: 1.5260, Val Loss: 1.3550, Acc: 0.4030, Val Acc: 0.5170
Epoch 60, Loss: 1.3523, Val Loss: 1.1750, Acc: 0.4634, Val Acc: 0.5852
Epoch 70, Loss: 1.1915, Val Loss: 1.0157, Acc: 0.5160, Val Acc: 0.6534
Epoch 80, Loss: 1.0505, Val Loss: 0.8633, Acc: 0.5672, Val Acc: 0.7188
Epoch 90, Loss: 0.9302, Val Loss: 0.7713, Acc: 0.6340, Val Acc: 0.7159
Epoch 100, Loss: 0.8401, Val Loss: 0.7044, Acc: 0.6581, Val Acc: 0.7557
Epoch 110, Loss: 0.7645, Val Loss: 0.6560, Acc: 0.6780, Val Acc: 0.7557
Epoch 120, Loss: 0.6991, Val Loss: 0.5886, Acc: 0.7157, Val Acc: 0.7756
Epoch 130, Loss: 0.6321, Val Loss: 0.5542, Acc: 0.7420, Val Acc: 0.7699
Epo

...

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.6839, Val Loss: 2.5622, Acc: 0.0924, Val Acc: 0.0966
Epoch 10, Loss: 2.4654, Val Loss: 2.4187, Acc: 0.1613, Val Acc: 0.3011
Epoch 20, Loss: 2.1729, Val Loss: 2.0880, Acc: 0.2331, Val Acc: 0.3011
Epoch 30, Loss: 1.9128, Val Loss: 1.8020, Acc: 0.2985, Val Acc: 0.3949
Epoch 40, Loss: 1.6874, Val Loss: 1.5612, Acc: 0.3397, Val Acc: 0.4205
Epoch 50, Loss: 1.5019, Val Loss: 1.3327, Acc: 0.4144, Val Acc: 0.5426
Epoch 60, Loss: 1.3173, Val Loss: 1.1409, Acc: 0.4542, Val Acc: 0.6193
Epoch 70, Loss: 1.1844, Val Loss: 0.9823, Acc: 0.5338, Val Acc: 0.6506
Epoch 80, Loss: 1.0313, Val Loss: 0.8425, Acc: 0.5814, Val Acc: 0.6932
Epoch 90, Loss: 0.9395, Val Loss: 0.7456, Acc: 0.6041, Val Acc: 0.7102
Epoch 100, Loss: 0.8122, Val Loss: 0.7466, Acc: 0.6766, Val Acc: 0.6875
Epoch 110, Loss: 0.7599, Val Loss: 0.7377, Acc: 0.6923, Val Acc: 0.6989
Epoch 120, Loss: 0.7415, Val Loss: 0.6650, Acc: 0.6937, Val Acc: 0.7216
Epoch 130, Loss: 0.7007, Val Loss: 0.6094, Acc: 0.7214, Val Acc: 0.7528
Epo

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Loss: 2.7336, Val Loss: 2.5606, Acc: 0.0703, Val Acc: 0.0912
Epoch 10, Loss: 2.4635, Val Loss: 2.4309, Acc: 0.1534, Val Acc: 0.2393
Epoch 20, Loss: 2.2016, Val Loss: 2.1306, Acc: 0.2358, Val Acc: 0.3020
Epoch 30, Loss: 1.9766, Val Loss: 1.8543, Acc: 0.2798, Val Acc: 0.3732
Epoch 40, Loss: 1.7540, Val Loss: 1.5762, Acc: 0.3303, Val Acc: 0.4672
Epoch 50, Loss: 1.5433, Val Loss: 1.3386, Acc: 0.4091, Val Acc: 0.5470
Epoch 60, Loss: 1.3392, Val Loss: 1.1367, Acc: 0.4858, Val Acc: 0.6011
Epoch 70, Loss: 1.1767, Val Loss: 0.9771, Acc: 0.5206, Val Acc: 0.6809
Epoch 80, Loss: 1.0504, Val Loss: 0.8506, Acc: 0.5966, Val Acc: 0.6980
Epoch 90, Loss: 0.9331, Val Loss: 0.7621, Acc: 0.6214, Val Acc: 0.7208
Epoch 100, Loss: 0.8676, Val Loss: 0.6853, Acc: 0.6435, Val Acc: 0.7379
Epoch 110, Loss: 0.7509, Val Loss: 0.6349, Acc: 0.6996, Val Acc: 0.7521
Epoch 120, Loss: 0.6745, Val Loss: 0.5887, Acc: 0.7180, Val Acc: 0.7749
Epoch 130, Loss: 0.6286, Val Loss: 0.5666, Acc: 0.7486, Val Acc: 0.7692
Epo

{0.05: {'test_accuracy': 0.83315649867374,
  'f1_score': 0.8308637093296243,
  'homogeneity': 0.8141059242287847,
  'nmi': 0.8181186599402427,
  'ari': 0.7172208226752821},
 0.060000000000000005: {'test_accuracy': 0.8201591511936339,
  'f1_score': 0.8161126123801405,
  'homogeneity': 0.8096019053065527,
  'nmi': 0.8145921231620598,
  'ari': 0.7062671719817598},
 0.07: {'test_accuracy': 0.8209549071618036,
  'f1_score': 0.8161785833741744,
  'homogeneity': 0.8067399040598993,
  'nmi': 0.8120419746911871,
  'ari': 0.7041854207446072},
 0.08000000000000002: {'test_accuracy': 0.8196286472148542,
  'f1_score': 0.8133218716156844,
  'homogeneity': 0.8050171970294876,
  'nmi': 0.8113834367711013,
  'ari': 0.702723960809965},
 0.09000000000000001: {'test_accuracy': 0.8225464190981432,
  'f1_score': 0.8170486219640237,
  'homogeneity': 0.8073525852989297,
  'nmi': 0.8132736260411777,
  'ari': 0.7061775795412505},
 0.1: {'test_accuracy': 0.8305039787798408,
  'f1_score': 0.8270866334632359,
  'h

In [2]:
zip_filename = "/kaggle/working/kaggle_working_dir.zip"

# Zip the entire working directory
shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', "/kaggle/working")

print(f"Zipped file saved at: {zip_filename}")

Zipped file saved at: /kaggle/working/kaggle_working_dir.zip
